In [3]:
from pytube import YouTube
from pathlib import Path

In [30]:
video_link = "https://www.youtube.com/watch?v=5dp_AKwv5vI"
filename='hp.mp4'
scene_name = filename.split('.')[0]


In [43]:
ROOT = Path(".").absolute().parent
VIDEOS = ROOT/"videos"
DATA = ROOT/"data"
SCENE = DATA/scene_name
SCENE.mkdir(exist_ok=True, parents=True)
(SCENE/"input").mkdir(exist_ok=True)
(SCENE/"distorted").mkdir(exist_ok=True)

## Downlaod youtube video

In [20]:

video = YouTube(video_link)
for stream in video.streams:
    print(stream)

<Stream: itag="18" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">
<Stream: itag="271" mime_type="video/webm" res="1440p" fps="30fps" vcodec="vp9" progressive="False" type="video">
<Stream: itag="137" mime_type="video/mp4" res="1080p" fps="30fps" vcodec="avc1.640028" progressive="False" type="video">
<Stream: itag="248" mime_type="video/webm" res="1080p" fps="30fps" vcodec="vp9" progressive="False" type="video">
<Stream: itag="136" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.4d401f" progressive="False" type="video">
<Stream: itag="247" mime_type="video/webm" res="720p" fps="30fps" vcodec="vp9" progressive="False" type="video">
<Stream: itag="135" mime_type="video/mp4" res="480p" fps="30fps" vcodec="avc1.4d401e" progressive="False" type="video">
<Stream: itag="244" mime_type="video/webm" res="480p" fps="30fps" vcodec="vp9" progressive="False" type="video">
<Stream: itag="134" mime_type="video/mp4" res="3

In [ ]:

video_path = video.streams[2].download(
    output_path=VIDEOS/"inputs", filename=filename)


In [25]:
video_path = str(VIDEOS/"inputs"/filename)

## Extract image from video
If you change the input video, plese change the parameter:
* between(t, time1, time2) in the following expression

In [47]:
!ffmpeg -i {video_path} -vf "select='between(t,0.3,5)',setpts=N/FRAME_RATE/TB" -vsync vfr -q:v 2 {SCENE}/input/image_%04d.jpg

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with clang version 12.0.0
  configuration: --prefix=/Users/drwho/anaconda3 --cc=arm64-apple-darwin20.0.0-clang --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55.  5.100 / 55.  5.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/Users/drwho/github/gau

## COLMAP SFM (Structure from Motion)
> It's a lot of hassle to install this nakedly on your machine, this docker file will suffice just fine

### Build docker command

Run a docker container with the following command:

In [44]:
print(
    f"""docker run -it --rm -v {DATA}:/root/data --name colmap colmap/colmap /bin/bash
""")
 

docker run -it --rm -v /Users/drwho/github/gaussian-splatting/data:/root/data --name colmap colmap/colmap /bin/bash



Inside the docker container, run these following commands, it might take a while

### Cmd 1: Feature extraction

In [45]:
print(f"""
colmap feature_extractor \
  --database_path /root/data/{scene_name}/distorted/database.db \
  --image_path /root/data/{scene_name}/input \
  --ImageReader.single_camera 1 \
  --ImageReader.camera_model SIMPLE_PINHOLE \
  --SiftExtraction.use_gpu false
""")


colmap feature_extractor   --database_path /root/data/hp/distorted/database.db   --image_path /root/data/hp/input   --ImageReader.single_camera 1   --ImageReader.camera_model SIMPLE_PINHOLE   --SiftExtraction.use_gpu false



### Cmd 2: Feature matching

In [48]:
print(f"""
colmap exhaustive_matcher \
  --database_path /root/data/{scene_name}/distorted/database.db \
  --SiftMatching.use_gpu false
""")


colmap exhaustive_matcher   --database_path /root/data/hp/distorted/database.db   --SiftMatching.use_gpu false



### Cmd 3: Bundle adjustment
> The default Mapper tolerance is unnecessarily large, decreasing it speeds up bundle adjustment steps.

In [53]:
print(f"""
mkdir /root/data/{scene_name}/distorted/sparse;
colmap mapper \
--database_path /root/data/{scene_name}/distorted/database.db \
--image_path /root/data/{scene_name}/input \
--output_path /root/data/{scene_name}/distorted/sparse \
--Mapper.ba_global_function_tolerance=0.000001
""")


mkdir /root/data/hp/distorted/sparse;
colmap mapper --database_path /root/data/hp/distorted/database.db --image_path /root/data/hp/input --output_path /root/data/hp/distorted/sparse --Mapper.ba_global_function_tolerance=0.000001



### Cmd 4: Image undistortion
> We need to undistort our images into ideal pinhole intrinsics.

In [54]:
print(f"""
mkdir -p /root/data/{scene_name}/distorted/sparse/0;
colmap image_undistorter \
--image_path /root/data/{scene_name}/input \
--input_path /root/data/{scene_name}/distorted/sparse/0 \
--output_path /root/data/{scene_name}/ \
--output_type COLMAP""")


mkdir -p /root/data/hp/distorted/sparse/0;
colmap image_undistorter --image_path /root/data/hp/input --input_path /root/data/hp/distorted/sparse/0 --output_path /root/data/hp/ --output_type COLMAP


Move all the `sparse` files to `sparse/0` folder

### Move the sparse files

In [55]:
SPARSE = SCENE/"sparse"
SPARSE0 = SCENE/"sparse"/"0"
SPARSE0.mkdir(exist_ok=True, parents=True)

for file in SPARSE.iterdir():
    if file.name == "0":
        continue
    # move to sparse/0
    file.rename(SPARSE0/file.name)